In [ ]:
import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI, MCSAPI

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("SWAN_MAIN_API"), environment=os.getenv("SWAN_ENVIRONMENT"))

api_key = os.getenv("MCS_API")
mcs_api = MCSAPI(api_key)


In [ ]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

In [ ]:
device = "G1ae.medium"
obj = [hardware for hardware in hardwares if hardware.name == device][0]
print(obj.id)
print([(hardware.name, hardware.id, hardware.region) for hardware in hardwares if hardware.name == device][0])

In [ ]:
from swan.object.source_uri import LagrangeSpace

lag = LagrangeSpace('0x1898015fCde650CDB05a69FB9EC70C77e2145206', 'NSFW', os.getenv("WALLET"), obj)
 
lag.get_space_info()

# Upload Directory to MCS
response = lag.generate_source_uri('swan_test', 'mar8s1', 'nsfw.json', mcs_client = mcs_api)

In [ ]:
print(lag.source_uri)

In [ ]:
import sys
sys.path.insert(0, '..')
 
import os
from dotenv import load_dotenv
load_dotenv()
 
from swan import SwanContract
 
pk = os.getenv('PK')
rpc = os.getenv('RPC')
 
c = SwanContract(pk, rpc)
 
# Test esimate lock revenue
r = c.estimate_payment(obj.id, 2)
print(r)

In [ ]:
r = c._approve_swan_token(22000000000000000000)
print(r)
 
r = c.lock_revenue('1', obj.id, 2)
print(r)

In [ ]:
# Retrieve List of Hardwares

# for hardware in hardwares:
#     if hardware.name == 'C1ae.medium':
#         print(hardware.region, hardware.price)
        
# Deploy task
result = swan_api.deploy_task(
                              cfg_name=device, 
                              region='California-US', 
                              start_in=60*40, duration=3600*1, 
                              job_source_uri=lag.source_uri, 
                              paid=22,
                              tx_hash="0xc7dc919dfa5cb3375fefb70b8ba5b96dc488890ac9aa8d58e261ca33d6af0cc1",
                              wallet_address=os.getenv('WALLET'),
                            )
print(result)
task_uuid = result['data']['task']['uuid']
print("Task UUID:", task_uuid)

In [ ]:
import json

# Check task info
while True:
    info = swan_api.get_deployment_info(task_uuid=task_uuid)
    if len(info['data']['jobs']) > 0:
        
        status = info['data']['jobs'][0]['status']
        print(status)
        
        job_res_uri = info['data']['jobs'][0]['job_result_uri']
        job_real_uri = info['data']['jobs'][0]['job_real_uri']
        print("Job Result URL: ", job_res_uri)
        print("Job Real URL: ", job_real_uri)
        
        # break
        if status == 'deploytok8s' or status == "Cancelled" or status == "Failed":
            break
        
    time.sleep(30)

In [ ]:
r = swan_api.get_real_url(task_uuid)
print(r)

In [29]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}
json_data = {
    'link': [
        'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s',
        'https://upload.wikimedia.org/wikipedia/commons/1/18/Ipfs-logo-1024-ice-text.png',
        'www.google.com',
        'www.pornhub.com',
    ],
}

response = requests.post(f"https://1j2muw1lae.dev2.crosschain.computer/link", headers=headers, json=json_data)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


{
    "result": [
        {
            "is_nsfw_image": true,
            "is_nsfw_link": false,
            "link": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYf_0-TvQ0WIDT0wFTAzdqD-c0y1mSfQjfOO0ydStXxA&s",
            "probability": 0.999750554561615
        },
        {
            "is_nsfw_image": false,
            "is_nsfw_link": false,
            "link": "https://upload.wikimedia.org/wikipedia/commons/1/18/Ipfs-logo-1024-ice-text.png",
            "probability": 0.999238133430481
        },
        {
            "is_nsfw_image": false,
            "is_nsfw_link": false,
            "link": "www.google.com",
            "probability": null
        },
        {
            "is_nsfw_image": null,
            "is_nsfw_link": true,
            "link": "www.pornhub.com",
            "probability": 100
        }
    ]
}


In [ ]:
log_path = '../nsfw_detection.csv'

if not os.path.isfile(log_path):
    with open(log_path, 'w') as f:
        f.write('link,IS NSFW IMAGE,IS NSFW LINK,Probability\n')

with open(log_path, 'a') as f:
    for result in response.json()['result']:
        f.write(f"{result['link']},{result['is_nsfw_image']},{result['is_nsfw_link']},{result['probability']}\n")
